# Preprocess NOAA data.


In [28]:
import pandas as pd
import requests

pages = 8

for page in range(1, pages + 1):
    if (page == 1):
        r = requests.get(f"https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/earthquakes?minYear=2000")
    else:
        r = requests.get(f"https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/earthquakes?minYear=2000&page={page}")
    data = r.json()["items"]
    if (page == 1):
        all_data = data
    else:
        all_data.extend(data)

df = pd.json_normalize(all_data)

In [ ]:
df.to_csv("data/noaa_earthquakes_2000_2025.csv", index=False)

In [16]:
pd.set_option('display.max_columns', None)
print(df.columns)
df.head()

Index(['id', 'year', 'month', 'day', 'hour', 'minute', 'second',
       'locationName', 'latitude', 'longitude', 'eqDepth', 'eqMagnitude',
       'damageAmountOrder', 'eqMagMb', 'publish', 'damageAmountOrderTotal',
       'housesDamagedTotal', 'housesDamagedAmountOrderTotal', 'country',
       'regionCode', 'injuries', 'injuriesAmountOrder', 'housesDestroyed',
       'housesDestroyedAmountOrder', 'housesDamaged',
       'housesDamagedAmountOrder', 'eqMagMw', 'eqMagMs', 'injuriesTotal',
       'injuriesAmountOrderTotal', 'housesDestroyedTotal',
       'housesDestroyedAmountOrderTotal', 'deaths', 'deathsAmountOrder',
       'damageMillionsDollars', 'eqMagMl', 'deathsTotal',
       'deathsAmountOrderTotal', 'damageMillionsDollarsTotal',
       'tsunamiEventId', 'intensity', 'volcanoEventId', 'area', 'missing',
       'missingAmountOrder', 'missingTotal', 'missingAmountOrderTotal',
       'eqMagUnk'],
      dtype='object')


,id,year,month,day,hour,minute,second,locationName,latitude,longitude,eqDepth,eqMagnitude,damageAmountOrder,eqMagMb,publish,damageAmountOrderTotal,housesDamagedTotal,housesDamagedAmountOrderTotal,country,regionCode,injuries,injuriesAmountOrder,housesDestroyed,housesDestroyedAmountOrder,housesDamaged,housesDamagedAmountOrder,eqMagMw,eqMagMs,injuriesTotal,injuriesAmountOrderTotal,housesDestroyedTotal,housesDestroyedAmountOrderTotal,deaths,deathsAmountOrder,damageMillionsDollars,eqMagMl,deathsTotal,deathsAmountOrderTotal,damageMillionsDollarsTotal,tsunamiEventId,intensity,volcanoEventId,area,missing,missingAmountOrder,missingTotal,missingAmountOrderTotal,eqMagUnk
0,5551,2000,1,3,22.0,34.0,12.6,INDIA-BANGLADESH BORDER: MAHESHKHALI,22.132,92.771,33.0,4.6,1.0,4.6,True,1.0,100.0,2.0,INDIA,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5552,2000,1,11,23.0,43.0,56.4,CHINA: LIAONING PROVINCE,40.498,122.994,10.0,5.1,3.0,4.9,True,3.0,8800.0,4.0,CHINA,30,30.0,1.0,3600.0,4.0,8800.0,4.0,5.1,4.7,30.0,1.0,3600.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5553,2000,1,14,23.0,37.0,7.8,CHINA: YUNNAN PROVINCE: YAOAN COUNTY,25.607,101.063,33.0,5.9,4.0,5.4,True,4.0,NaN,NaN,CHINA,30,2528.0,4.0,41000.0,4.0,NaN,NaN,5.9,5.9,2528.0,4.0,41000.0,4.0,5.0,1.0,73.5,5.5,7.0,1.0,73.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5554,2000,2,2,22.0,58.0,1.5,"IRAN: BARDASKAN, KASHMAR",35.288,58.218,33.0,5.3,2.0,5.1,True,2.0,300.0,3.0,IRAN,140,15.0,1.0,100.0,2.0,100.0,2.0,5.3,5.3,15.0,1.0,100.0,2.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5555,2000,2,7,19.0,34.0,57.0,SOUTH AFRICA; SWAZILAND: MBABANE-MANZINI,-26.288,30.888,5.0,4.5,1.0,4.5,True,1.0,NaN,NaN,SOUTH AFRICA,10,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# combine housesDamagedTotal and housesDamaged columns
print(df["housesDamagedTotal"])
print(df["housesDamaged"])
# if housesDamagedTotal is different than housesDamaged, take an average
df["housesDamagedTotal"] = df["housesDamagedTotal"].fillna(df["housesDamaged"])
df["housesDamaged"] = df["housesDamaged"].fillna(df["housesDamagedTotal"])
df["housesDamagedTotal"] = df["housesDamagedTotal"].where(df["housesDamagedTotal"] == df["housesDamaged"], (df["housesDamagedTotal"] + df["housesDamaged"]) / 2)
print(df["housesDamagedTotal"])

0        100.0
1       8800.0
2          NaN
3        300.0
4          NaN
         ...  
1420       NaN
1421       NaN
1422       NaN
1423    4328.0
1424       NaN
Name: housesDamagedTotal, Length: 1425, dtype: float64
0          NaN
1       8800.0
2          NaN
3        100.0
4          NaN
         ...  
1420       NaN
1421       NaN
1422       NaN
1423    4328.0
1424       NaN
Name: housesDamaged, Length: 1425, dtype: float64
0        100.0
1       8800.0
2          NaN
3        200.0
4          NaN
         ...  
1420       NaN
1421       NaN
1422       NaN
1423    4328.0
1424       NaN
Name: housesDamagedTotal, Length: 1425, dtype: float64


In [ ]:
df["housesDamagedTotal"] = df["housesDamagedTotal"].fillna(df["housesDamaged"]).fillna(0)
df = df.drop(columns=["housesDamaged"])
df.head()